In [63]:
from openai import OpenAI
import os
from typing import Generator
import speech_recognition as sr
from pydub import AudioSegment

os.system("sh ../.env")
YOUR_API_KEY = os.environ.get("YOUR_API_KEY")
print(YOUR_API_KEY)
MODEL_NAME_HERE = "llama-3.1-sonar-large-128k-online"

with open("comments.txt","w") as f:
    pass
with open("summary.txt","w") as f:
    pass

import subprocess
import os

def convert_mp4_to_wav(input_file, output_file):
    """
    Convert an MP4 video file to a WAV audio file using FFmpeg.
    
    :param input_file: The input .mp4 file path.
    :param output_file: The output .wav file path.
    """
    # Use FFmpeg command to convert MP4 to WAV
    command = [
        "ffmpeg",
        "-i", input_file,      # Input file
        "-vn",                  # No video
        "-acodec", "pcm_s16le",  # Audio codec
        "-ar", "44100",        # Sample rate
        "-ac", "2",            # Number of audio channels
        output_file            # Output file path
    ]
    
    # Run the command
    subprocess.run(command, check=True)
    print(f"Converted {input_file} to {output_file}")

# Example usage:
input_mp4_file = "/home/fedora/data/oj1.mp4"
output_wav_file = "output.wav"

if not "output.wav" in os.listdir():convert_mp4_to_wav(input_mp4_file, output_wav_file)


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


In [64]:



CHUNK_SIZE = 60000  # milliseconds



def speech_to_text(audio_path: str) -> Generator[str, None, None]:
    audio = AudioSegment.from_wav(audio_path)
    num_chunks = len(audio) // CHUNK_SIZE + (1 if len(audio) % CHUNK_SIZE > 0 else 0)
    try:
        os.mkdir(
        "temp"
        )
    except:
        os.system("temp/*")

    td= "temp"
    for i in range(num_chunks):
        start_time = i * CHUNK_SIZE
        end_time = start_time + CHUNK_SIZE
        chunk = audio[start_time:end_time]
        chunk_filename = f"{td}/chunk_{i}.wav"
        chunk.export(chunk_filename, format="wav")

        yield _recognise_text(chunk_filename)


def _recognise_text(audio_path: str) -> str:
    recogniser = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        data = recogniser.record(source)

    text = recogniser.recognize_google(data)
    return text


In [65]:
text = speech_to_text("output.wav")



In [66]:
print(text)

<generator object speech_to_text at 0x7fb73485a680>


In [67]:
import tiktoken

def chunk_trancript(text):
# Load GPT-4 tokenizer
    encoding = tiktoken.encoding_for_model("gpt-4o")
    max_tokens = 8000
    tokens = encoding.encode(text)
    chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]

    
    text_chunks = [encoding.decode(chunk) for chunk in chunks]
    return text_chunks
    


In [68]:
def gen_message(chunks):
    prev_chunk = None
    prev_summarization = None
    messages = []
    for i in range(len(chunks)):
        current_chunk = chunks[i]
        current_message = [
            {
                "role": "system",
                "content": (
                    "You are a lawyer who summarises the transcript of legal cases. Be concise, accurate and understandable."
                ),
            },
            {
                "role": "user",
                "content": (
                    f"Summarize the current body of text, which is a chunk of a larger document '{current_chunk}'. Your summarization should contain all the important details in a readable form. Be concise. Here is the previus"
                ),
            },
        ]
        messages.append(current_message)
    return messages
def gen_single_message(chunk,text_response):
    postfix = "Here is the previous response '{text_response}'" if  text_response  else ""
    current_message = [
        {
            "role": "system",
            "content": (
                "You are a lawyer who summarises the transcript of legal cases. Be concise, accurate and understandable. Do not include an introduction or a conclusion."
            ),
        },
        {
            "role": "user",
            "content": (
                f"Summarize the current body of text, which is a chunk of a larger document '{chunk}'. Your summarization should contain all the important details in a readable form. Be concise."+postfix
            ),
        },
    ]
    return current_message

def gen_lawyer_message(summaries):
    current_message = [
        {
            "role": "system",
            "content": (
                "You are a lawyer who analyzes snippets court cases based on their transcripts/summaries. Provide insightful and interesting commentary on the proceedings of the cases. Keep your comments short and to the point, do not give an introduction or conclusion."
            ),
        },
        {
            "role": "user",
            "content": (
                f'Analyze the following proceedings and give unique and interesting opinions on it. Clarify any points that may not be accessible to a general audience."{' '.join(summaries)}"'
            ),
        },
    ]
    return current_message

In [69]:
def summarize_chunk(chunk,text_response):
    message = gen_single_message(chunk,text_response)

    response = client.chat.completions.create(
        model=MODEL_NAME_HERE,
        messages=message,
    )
    text_response = response.choices[0].message.content.strip()
    return text_response
def comment_on_summaries(outputs):
    message = gen_lawyer_message(outputs)

    response = client.chat.completions.create(
        model=MODEL_NAME_HERE,
        messages=message,
    )
    text_response = response.choices[0].message.content.strip()
    return text_response

while True:
    text_chunks = chunk_trancript(next(text))

    client = OpenAI(api_key=YOUR_API_KEY, base_url="https://api.perplexity.ai")

    outputs,summaries = [],[]
    text_response = None
    # chat completion without streaming
    for chunk in text_chunks:
        text_response = summarize_chunk(chunk,text_response)
        with open("summary.txt","a") as f:
            f.write(text_response)
        outputs.append(text_response)
        lawyer_comment = comment_on_summaries(outputs)
        with open("comments.txt","a") as f:
            f.write(lawyer_comment)

        summaries.append(lawyer_comment)

sh: line 1: temp/chunk_0.wav: Permission denied


UnknownValueError: 